# BIG DATA ANALYTICS  PROJECT  “Opinion Mining  with Spark”  REPORT
**Fabrice ZAPFACK, Sofia HAFDANI, Svetlana Smagina**

** *!!! Don't run this codes as they are intended to be run in a pyspark context* **

*In this project we faced a classification problem, the being to predict if a particular text correspond to a **good** or a **bad** review.The problem therefore correspond to a sentiment analysis problem.*

To resolve this problem, we are given 2 datasets :
1.  A set of 25,000 documents that contain labeled reviews either as positive or negative (50%-50%). This will be used for TRAINING. 
2. Another set of 25.000 documents containing unlabeled reviews that we need to assign labels to them. This set will be used for TESTING. 

# First approach : Resilient Distributed Datasets (RDDs)

As a first step in this project, we decided to implement our models using spark in order to parallelize the task.

## Data import

The data files were availables for the course website. A python script, **< loadFiles.py>** was given to be able to import those files.
* To load the training data, the path of the directory needs to be passed as argument and it returns a python list containing the 25000 text reviews and a target vector (numpy) containing the label of each text (1 if positive an 0 if negative)
* To load the test data, the path of the directory needs to be passed as argument and it returns a python list containing the 25000 text reviews and a list containing the name each text file

In [ ]:
import loadFiles as lf
data,Y=lf.loadLabeled("./data/train")
test,names=lf.loadUknown('./data/test')

## Text Preprocessing

### Punctuation removal
The first step of the preprocessing consist of removing the punctuations (to help plitting the documument in words). For that we first used for loop to replace punctuations by " ".

In [ ]:
for w in ['.',',','--',':','!','?','(',')','"','/','<','>']:
	doc=doc.replace(w,' ')

We used instead regular expression from the package **< re >**. Depending of the features we wanted to extract, we conserved either alpha-numeric characters (bag of words appraoches) or alphabetical (word2vec)

In [ ]:
import re
doc = re.sub("[^a-zA-Z]"," ", doc) #alphabetical characters conserved
doc = re.sub("[^0-9a-zA-Z]"," ", doc) #alpha-numeric characters conserved

We have decided to remove all the punctuations even we know that somitimes they are linked to sentiments ("!!!", ":-)", ":(", ...). Also some html tags where presents in the texts. It is recommanded to remove them using powerful packages like **<beautifulSoup>** but we considered it was not worth to do it as that packages were not present in the machines cluster and removed those tag simply by adding words like **'html', 'br', ...** in the stopwords list.

### Document spliting and lowercase
The documents were then splitted in a list of words and each of those words were transformed to lowercase.

In [ ]:
words=doc.strip().split(' ')
words = [w.lower() for w in words]

### Stopwords removal
The stopwords used in this project were a concatenation of self-made stopwords and the ones given in the nltk package. We also decided to remove words which contains less than 3 characters.
** the stopwords where removed only for the bag-of-words approaches **

In [ ]:
# stop-words based on the previous lab we add nltk stopwords
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
			'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
                'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
                'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
                'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
                'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
                'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
                'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
                'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
                'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
                'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
                'than', 'too', 'very', 's', 't', 'can','could', 'will','would', 'just', 'don', 'should', 'now','films',
                'film','movies','movie','br','http','also','seem', ' ', '']
stop_words.extend([w.encode('ascii','ignore') for w in stopwords.words("english")])
stop_words.extend([w for w in words if len(w)<3]) 
stop_words = list(set(stop_words)) 

### Stemming 
We also used porter stemmenr, implemented in nltk for stemming. However stemming was removed in our final submission as it decreases the performance obtained by cross validation

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
words = [stemmer.stem(unicode(w, "utf-8")) for w in words] 

### N-grams
We decided to limit our word with 3-grams because believe that bigger features will capture more noise than information and also because the number of features was already very big when using 3-grams.

In [ ]:
bigrams = zip(words,words[1:])
trigrams = zip(words,words[1:],words[2:])
bigrams = [" ".join(gram) for gram in bigrams if not any(i in stop_words for i in gram)]
trigrams = [" ".join(gram) for gram in trigrams if not any(i in stop_words for i in gram)]
words = [w for w in words if w not in stop_words and w not in punctuation]
words.extend(bigrams)
words.extend(trigrams)

### Dimension Reduction

#### Latent Dirichlet allocation
We fitted an LDA to reduce dimensionality to the most relevant topics: 
Latent Dirichlet allocation (LDA) is a generative probabilistic model of a corpus. The basic idea is that documents are represented as random mixtures over latent topics, where each topic is characterized by a distribution over words.


## Features Extraction

### Bag of words approach

#### Presence/absence of word 
This is a simple approach where a document is represented by a vector (sparse representation) where each column is 0 or 1 (1 if the corresponding word appears in the document 0 if not). We used the code provided in the function **< createBinaryLabeledPoint >**.

#### TF-IDF 
This approach is similar to previous one. There differnce is that instead of representing a document by 0/1 vector, it is now represented by vector containing the tf-idf of each words in the document.
 - First we compute the dictionnary 
 - For each document 
     - we split the document in a list of words (tokenization)
     - we count the number of occurrence of each words using **< HashingTF >** (TF)
     - we multiply the tf by the idf (inverse proportionnal of the occurence of the word in the corpus)

In [ ]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
dataRDD=sc.parallelize(data,numSlices=num_slices)
lists=dataRDD.map(doc2words)
# create dict
all=[]
for l in lists.collect():
	all.extend(l)
dict=set(all)

# TF-IDF
print "len dict {} ".format(len(dict)) 
hashingTF = HashingTF(numFeatures=len(dict))
tf = hashingTF.transform(lists)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf).collect()

### More Feature Engineering with: Word2Vec method

We first need to perform tokenization word2vec is suppose to perform better without stopword removal.
We then use the lists of words to train a word2vec model implemented in < mllib>.
As reference, we learned that: 
Word2vec is a group of related models that are used to produce so-called word embeddings. These models are shallow, two-layer neural networks, that are trained to reconstruct linguistic contexts of words: the network is shown a word, and must guess at which words occurred in adjacent positions in an input text. The order of the remaining words is not important (bag-of-words assumption) (Mikolov, Tomas; et.al (2013). "Distributed representations of words and phrases and their compositionality") 
After training, word2vec models can be used to map each word to a vector of typically several hundred elements, which represent that word's relation to other words.

In [ ]:
from pyspark.mllib.feature import Word2Vec
dataRDD=sc.parallelize(data,numSlices=num_slices)
sentencesRDD = dataRDD.map(lambda x: doc2words(unicode(x,errors='ignore')))
print "Start word2vec"
start = time.time()
word2vec = Word2Vec().setVectorSize(200).setSeed(42).setMinCount(50)
model = word2vec.fit(sentencesRDD)
words_vect=model.getVectors()
end = time.time()
print "End word2vec learning. duration {} s".format(end-start)

#### Comments on the word2vec feature engineering method


It seems that this model does not capture well the sentiment in the text as the synonyms found are not very relevant. We therefore tried a second time using the words 'comment', 'comments', 'user', 'users', 'imdb', 'movie', 'movies', 'film', 'films' as stopwords but without success.

The implementation of word2vec used doesn't directly give the matrix containing the vector representation of the words. To compute it, we first computed the dictionnary of the corpus (excluding the stopwords), and each term in the dictionary was then transformed in the word vector space.


## Build a dictionnary

In [ ]:
start = time.time()
lists=dataRDD.map(doc2words).collect()
all=[]
for l in lists:
	all.extend(l)
dictionary=set(all)
print "Number of elements in dictionnary %d" % (len(dictionary))
words_vectors = {}
test = {}
for x in dictionary:
	print x
	test[x] = x
	words_vectors[x] = model.transform(x)
end = time.time()
print "time {}".format(end-start)


---

## Model
In this project we tested the 3 classification models implemented in < mllib > :
- Naives Bayes
- Linear SVM
- Logistic regression




## Performance metrics
To evaluate the performance of thes algorithms, we decide to use ** accuracy** as criteria. We have decided to use only this parameter because it is the only one that is used for evaluation by the examinator. We could have used other ones as precision, recall, auc, f2-score, confusion matrix ...

### Model Evaluation
The models where evualuted using cross-validation. To do that we performed a shuffle split on the labelled training data because the initial data set was ordored.

In [ ]:
## Sample of the code 
from  pyspark.mllib.classification import NaiveBayes
from  pyspark.mllib.classification import SVMWithSGD
from  pyspark.mllib.classification import LogisticRegressionWithLBFGS
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import accuracy_score

### cross-validaton
cv = ShuffleSplit(len(tfidf), n_iter=3, test_size=0.3, random_state=42)
models = [NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD]
scores = {model.__name__: [] for model in models}
for i_cv, i_temp in enumerate(cv):
    i_train = i_temp[0]
	i_test = i_temp[1]
	data_train = [data[i] for i in i_train]
	Y_train = [Y[i] for i in i_train]
	data_test = [data[i] for i in i_test]
	Y_test = [Y[i] for i in i_test]
    ...
    ...
    for model in models:
### Model training
		model_trained=model.train(labeledRDD)
		mb=sc.broadcast(model_trained)
        
### Model testing
		Y_pred = model_trained.predict(testRDD2).collect()
		score = accuracy_score(Y_test, Y_pred)
		scores[model.__name__].append(score)
for key, value in scores.items():
	print "%s : mean=%.5f, std=%.5f " %(key, np.mean(value), np.std(value))

## Parameters tuning
We perfomed gridsearch in order to optimize hyperparameters, the loop we implemented returns scores that allow us to choose the best hyperparameters for each model.

In [ ]:
### Gridsearch Implemented on spark

grids = [ [{'lambda_':1.0 },{'lambda_':10.0 }], 
[{"iterations":100, "initialWeights":None, "regParam":0.01, 'regType':'l2', 'intercept':False, 'corrections':10, 'tolerance':0.0001, 'validateData':True, 'numClasses':2}],
[{'iterations':100, 'step':1.0, 'regParam':0.01, 'miniBatchFraction':1.0, 'initialWeights':None, 'regType':'l2', 'intercept':False, 'validateData':True}] 
]
### Each model has its own parameters: 

-SVM => terations=100, step=1.0, regParam=0.01, miniBatchFraction=1.0, initialWeights=None, regType='l2', intercept=False, validateData=True

-LR => terations=100, initialWeights=None, regParam=0.01, regType='l2', intercept=False, corrections=10, tolerance=0.0001, validateData=True, numClasses=2

-NaiveBayes => lambda_



# Second approach : 

# Models

-Random forests

-Extra trees classifier

-SGDClassifier with l1, l2, and elastic net penalizations

-DecisionTreeClassifier

-Ada Boost classifier

-Voting Classifier

### Parallelization of non-distributed models
We also wanted to fit more advanced models, however, due to the lack of resources on Spark's machine learning libraries we decided to implement them using scikit learn. 
The code describing more thouroughtly this approach is available in our ipython notebook 'second.ipynb '. 
We wanted to reduce the dimensionality of the data using an LDA implemented on scikit learn. The code ran smoothly locally but did not work on the cluster because it required an updated version of scikit learn (0.17). For this reason, we tried to use the lda package "lda 1.0.3". 

### Feature Selection:

In this second approach, feature selection was partly induced by the parameter tuning of the 'Countvectorizer' function from scikit learn:
-Max_df parameter: allows to ignore terms that occured in too many documents
-Min_df parameter: allows to ignore terms that occured in too few documents
-Max_features parameter: allows to perform more feature selection

In [ ]:
### Sample of our code 

from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.grid_search import RandomizedSearchCV 
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from scipy.stats import randint as sp_randint
from numpy.random import rand
from sklearn.naive_bayes import GaussianNB




n_iter_search = 3
param_dist = {"rf__n_estimators":sp_randint(1,100),
              "rf__max_features":sp_randint(1,20),
               "rf__max_depth" :sp_randint(1,20),
              "rf__max_features":sp_randint(1,20)
            }

clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
class Classifier(BaseEstimator):
    
    def __init__(self):
        pipeline= Pipeline([
              
             ('rf',RandomForestClassifier(n_jobs=-1) )    
        ])
        
        #('vc', VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('nb', clf3)], voting='soft',weights=))
        self.clf = RandomizedSearchCV(pipeline, param_distributions= param_dist, n_iter=n_iter_search)    
    
    def fit(self, X, y):
        self.clf.fit(X, y)
        report(self.clf.grid_scores_)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)
       


### Results

1. RandomForestClassifier--> we tuned our parameters with a randomized grid search, obtained the best hyperparameters but the accuracy did not exceed 0.81

2. We also tried other models such as ExtraTreesClassifier and DecisionTreeClassifier, however even with a thouroughtly tuning the accuracy did not exceed 0.79
3. Finally we performed a voting classifier. This method is one of the ensemble methods that performs soft voting and majority voting for many estimates. We chose the ‘soft’ voting method which, predicts the class label based on the argmax of the sums of the predicted probalities, which is recommended for an ensemble of well-calibrated classifiers.
The method improved the performance as expected because it grasps the wiknesses of every estimor. 

### Global conclusions for this second approach: 
From our interpretation of the results, linear models perform better in sentiment analysis than "tree" models. 


## Prediction

Our final predictions are summarized in the file named results.csv and the code related to the final cross validation 
is described in the pred_final.py file. 
